In [1]:
%cd ..

/home/eli/AnacondaProjects/categorical_bpl


In [2]:
import argparse
import collections
import pyro
import torch
import numpy as np
import data_loader.data_loaders as module_data
import model.model as module_arch
from parse_config import ConfigParser
from trainer import Trainer

In [3]:
%matplotlib inline

In [4]:
# pyro.enable_validation(True)
# torch.autograd.set_detect_anomaly(True)

In [5]:
# fix random seeds for reproducibility
SEED = 123
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(SEED)

In [6]:
Args = collections.namedtuple('Args', 'config resume device')
config = ConfigParser.from_args(Args(config='omniglot_config.json', resume=None, device=None))

In [7]:
logger = config.get_logger('train')

In [8]:
# setup data_loader instances
data_loader = config.init_obj('data_loader', module_data)
valid_data_loader = data_loader.split_validation()


Extracting data/omniglot-py/images_background.zip to data/omniglot-py



Extracting data/omniglot-py/images_evaluation.zip to data/omniglot-py
Files already downloaded and verified
Files already downloaded and verified


In [9]:
# build model architecture, then print to console
model = config.init_obj('arch', module_arch)

In [10]:
optimizer = pyro.optim.ReduceLROnPlateau({
    'optimizer': torch.optim.Adam,
    'optim_args': {
        "lr": 1e-3,
        "weight_decay": 0,
        "amsgrad": True
    },
    "patience": 50,
    "cooldown": 25,
    "factor": 0.1,
    "verbose": True,
})

In [11]:
trainer = Trainer(model, [], optimizer, config=config,
                  data_loader=data_loader,
                  valid_data_loader=valid_data_loader)

In [12]:
trainer.train()

Train Epoch: 1 [512/17352 (3%)] Loss: 1722.337158
Train Epoch: 1 [10068/17352 (58%)] Loss: 1082.017423
Train Epoch: 1 [16939/17352 (98%)] Loss: 973.377068
    epoch          : 1
    loss           : 1127.89618291453
    val_loss       : 947.8297191672776
    val_log_likelihood: -816.1482405239784
    val_log_marginal: -893.9770626010408
Train Epoch: 2 [512/17352 (3%)] Loss: 970.833618
Train Epoch: 2 [10382/17352 (60%)] Loss: 938.529755
Train Epoch: 2 [17263/17352 (99%)] Loss: 840.577303
    epoch          : 2
    loss           : 878.7033057891047
    val_loss       : 834.993852410643
    val_log_likelihood: -755.2571964150849
    val_log_marginal: -798.0746582828151
Train Epoch: 3 [512/17352 (3%)] Loss: 868.070679
Train Epoch: 3 [10694/17352 (62%)] Loss: 777.743218
Train Epoch: 3 [17153/17352 (99%)] Loss: 830.023239
    epoch          : 3
    loss           : 795.8705032507338
    val_loss       : 768.0807685528231
    val_log_likelihood: -720.1796183177313
    val_log_marginal: -749.